In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from wordcloud import WordCloud 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
import spacy
from collections import Counter

nlp = spacy.load('en_core_web_sm')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

In [ ]:
data=pd.read_csv("Precily_Text_Similarity.csv")
data.head(1)

## Preprocessing the text in the dataset

In [ ]:
def remove_punc(copydata):
    pattern = r'[' + string.punctuation + ']'
    copydata['text1']=data['text1'].map(lambda m:re.sub(pattern," ",m))
    copydata['text2']=data['text2'].map(lambda m:re.sub(pattern," ",m))
    return copydata

In [ ]:
def lower(df):
    df['text1'] = df['text1'].map(lambda row: row.lower())
    df['text2'] = df['text2'].map(lambda row: row.lower())
    return df

In [ ]:
from nltk.tokenize import word_tokenize
def tokenize(df):
    df['text1'] = df['text1'].apply(word_tokenize)
    df['text2'] = df['text2'].apply(word_tokenize)
    return df

In [ ]:
#on tokenized data
def rm_SW(df):
    sw=nltk.corpus.stopwords.words('english')
    df['text1'] = df['text1'].apply(lambda x: [item for item in x if item not in sw])
    df['text2'] = df['text2'].apply(lambda x: [item for item in x if item not in sw])
    return df

In [ ]:
def rm_digits(df):
    df['text1'] = df['text1'].apply(lambda row: re.sub('\d.*','',row))
    df['text1'] = df['text1'].apply(lambda row: re.sub('\d.*','',row))
    return df

In [ ]:
#ON LEMETIZE DATA
def lmtz(df):
    lm = WordNetLemmatizer()
    df['text1']=df['text1'].apply(lambda row: [lm.lemmatize(word) for word in row])
    df['text2']=df['text2'].apply(lambda row: [lm.lemmatize(word) for word in row])
    return df

## Word Embeddings from pre trained model -> Hugging face

In [ ]:
from sentence_transformers import SentenceTransformer,util
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
sentence_pairs = df[["text1", "text2"]].values
print(sentence_pairs)

In [ ]:
embeddings1, embeddings2 = [], []
for i in range(len(df)):
  new_embedding1 = model.encode(df['text1'][i])
  embeddings1.append(new_embedding1)
  new_embedding2 = model.encode(df['text2'][i])
  embeddings2.append(new_embedding2)

In [ ]:
cosine_similarities = cosine_similarity(embeddings1, embeddings2)